In [ ]:
# Cloning Github Repository and navigate into project folder
!git clone https://github.com/francmeister/Masters-Research-Project.git

In [ ]:
cd Masters-Research-Project/User-Association-Federated-Learning

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
import numpy as np
import random
import os
import time
import gym
import time
import threading
import multiprocessing
from eMBB_UE import eMBB_UE
from URLLC_UE import URLLC_UE
from SBS import SBS
from Communication_Channel import Communication_Channel
from numpy import interp
import math
from NetworkEnv import NetworkEnv
from global_entity import GLOBAL_ENTITY
from custom_barrier import CustomBarrier
from DNN import DNN
from DNN_training_memory import DNN_TRAINING_MEMORY
from global_entity import GLOBAL_ENTITY

In [2]:

#Create a Square grid of overlapping Access Points (3 APs) - 0 to 100 km both x and y

x_grid = 5000
y_grid = 5000
access_point_radius = 4000

num_embb_users = 12
num_urllc_users = 0
all_users = []
user_count = 1
embb_user_count = 1
urllc_user_count = 1
access_point_count = 1
access_points = []
num_access_points = 3
num_users = num_embb_users+num_urllc_users
num_input_features_per_user = 2
num_input_features = num_users*num_input_features_per_user
num_output_features = num_users
max_samples = 40
comm_channel = Communication_Channel(1)

In [3]:

for x in range(0,num_embb_users):
   embb_user = eMBB_UE(embb_user_count,user_count,100,600)
   all_users.append(embb_user)
   embb_user_count+=1
   user_count+=1

for x in range(0,num_urllc_users):
   urllc_user = URLLC_UE(urllc_user_count,user_count,100,600)
   all_users.append(urllc_user)
   urllc_user_count+=1
   user_count+=1

for x in range(0,num_access_points):
   access_point = SBS(access_point_count,num_access_points, num_input_features, num_output_features)
   access_points.append(access_point)
   access_point_count+=1

access_point_coordinates = []
for access_point in range(0, num_access_points):
   x_coord = random.randint(0, x_grid)
   y_coord = random.randint(0, y_grid)
   access_point_coordinates.append((x_coord,y_coord))

user_coordinates = []
for user in range(0, num_users):
   x_coord = random.randint(0, x_grid)
   y_coord = random.randint(0, y_grid)
   user_coordinates.append((x_coord,y_coord))

coord_index = 0
for access_point in access_points:
   access_point.set_coordinates(access_point_coordinates[coord_index])
   coord_index+=1

coord_index = 0
for user in all_users:
   user.set_coordinates(user_coordinates[coord_index])
   user.calculate_distances_from_access_point(access_point_coordinates,access_point_radius)
   coord_index+=1


for access_point in access_points:
   access_point.find_users_within_distance_radius(access_point_radius, all_users)


In [4]:
global_entity = GLOBAL_ENTITY(num_access_points)
global_entity.initialize_global_model(num_input_features,num_output_features)
global_memory = global_entity.initialize_global_memory(max_samples,num_users,num_input_features_per_user,num_access_points)
initial_user_associations = global_entity.perform_random_association(all_users)
#print(initial_user_associations)

In [7]:
for access_point in access_points:
   access_point.get_all_users(all_users)
   access_point.initialize_DNN_model(global_entity.global_model)
   access_point.acquire_global_memory(global_entity.global_memory)

[(array([1.        , 0.03887153, 2.        , 0.46226521, 3.        ,
       0.54421467]), array([0.71365581, 0.49014707, 0.2311162 ]), 0), (array([1.        , 0.89032033, 2.        , 0.2508279 , 3.        ,
       0.77845147]), array([0.49698885, 0.81685773, 0.89380255]), 1), (array([1.        , 0.10733734, 2.        , 0.51750087, 3.        ,
       0.19711612]), array([0.48568879, 0.11163281, 0.6008728 ]), 2), (array([1.        , 0.55994506, 2.        , 0.68391817, 3.        ,
       0.93808842]), array([0.70028848, 0.40430973, 0.28989028]), 1), (array([1.        , 0.27288044, 2.        , 0.14091557, 3.        ,
       0.77679398]), array([0.45959603, 0.28937989, 0.72820926]), 4), (array([1.        , 0.49248015, 2.        , 0.35969382, 3.        ,
       0.52211483]), array([0.32340303, 0.79840575, 0.26461144]), 3), (array([1.        , 0.40185714, 2.        , 0.06083432, 3.        ,
       0.50111864]), array([0.55256955, 0.72441219, 0.91338162]), 4), (array([1.        , 0.6980349 , 2

In [8]:
for access_point in access_points:
   access_point_users = []
   for user in all_users:
      for association in initial_user_associations:
         #print(association)
         if association[0] == user.user_label and association[1][0] == access_point.SBS_label:
            user.distance_from_associated_access_point = association[1][1]
            access_point_users.append(user)

   access_point.associate_users(access_point_users)

In [23]:
channel_rates = []
iterations = 500
for i in range(0,iterations):
   initial_user_associations = global_entity.perform_random_association(all_users)
   for access_point in access_points:
      access_point_users = []
      for user in all_users:
         for association in initial_user_associations:
            #print(association)
            if association[0] == user.user_label and association[1][0] == access_point.SBS_label:
               user.distance_from_associated_access_point = association[1][1]
               access_point_users.append(user)

      access_point.associate_users(access_point_users)
      for user in access_point.users:
         user.current_associated_access_point = access_point.SBS_label
         user.calculate_user_association_channel_gains()
         user.calculate_distance_from_current_access_point()
         user_association_channel_rate = user.calculate_achieved_user_association_channel_rate(comm_channel)
         channel_rates.append(user_association_channel_rate)

max_channel_rate = max(channel_rates)
print('max_channel_rate: ', max_channel_rate, ' bits/slot')
for access_point in access_points:
   access_point.max_user_channel_rate = max_channel_rate


max_channel_rate:  17.009014412712443  bits/slot


In [ ]:
class ReplayBuffer(object):

  def __init__(self, max_size=1e6):
    self.storage = []
    self.max_size = max_size
    self.ptr = 0

  def add(self, transition):
    if len(self.storage) == self.max_size:
      self.storage[int(self.ptr)] = transition
      self.ptr = (self.ptr + 1) % self.max_size
    else:
      self.storage.append(transition)

  def sample(self, batch_size):
    ind = np.random.randint(0, len(self.storage), size=batch_size)
    batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = [], [], [], [], []
    for i in ind:
      state, next_state, action, reward, done = self.storage[i]
      batch_states.append(np.array(state, copy=False))
      batch_next_states.append(np.array(next_state, copy=False))
      batch_actions.append(np.array(action, copy=False))
      batch_rewards.append(np.array(reward, copy=False))
      batch_dones.append(np.array(done, copy=False))
    return np.array(batch_states), np.array(batch_next_states), np.array(batch_actions), np.array(batch_rewards).reshape(-1, 1), np.array(batch_dones).reshape(-1, 1)


In [ ]:
class Actor(nn.Module):

  def __init__(self, state_dim, action_dim, max_action):
    super(Actor, self).__init__()
    self.layer_1 = nn.Linear(state_dim, 400)
    self.layer_2 = nn.Linear(400, 300)
    self.layer_3 = nn.Linear(300, action_dim)
    self.max_action = max_action

  def forward(self, x):
    x = F.relu(self.layer_1(x))
    x = F.relu(self.layer_2(x))
    x = self.max_action * torch.sigmoid(self.layer_3(x))
    #x = self.max_action * torch.tanh(self.layer_3(x))
    return x

In [ ]:
class Critic(nn.Module):

  def __init__(self, state_dim, action_dim):
    super(Critic, self).__init__()
    # Defining the first Critic neural network
    self.layer_1 = nn.Linear(state_dim + action_dim, 400)
    self.layer_2 = nn.Linear(400, 300)
    self.layer_3 = nn.Linear(300, 1)
    # Defining the second Critic neural network
    self.layer_4 = nn.Linear(state_dim + action_dim, 400)
    self.layer_5 = nn.Linear(400, 300)
    self.layer_6 = nn.Linear(300, 1)

  def forward(self, x, u):
    xu = torch.cat([x, u], 1)
    # Forward-Propagation on the first Critic Neural Network
    x1 = F.relu(self.layer_1(xu))
    x1 = F.relu(self.layer_2(x1))
    x1 = self.layer_3(x1)
    # Forward-Propagation on the second Critic Neural Network
    x2 = F.relu(self.layer_4(xu))
    x2 = F.relu(self.layer_5(x2))
    x2 = self.layer_6(x2)
    return x1, x2

  def Q1(self, x, u):
    xu = torch.cat([x, u], 1)
    x1 = F.relu(self.layer_1(xu))
    x1 = F.relu(self.layer_2(x1))
    x1 = self.layer_3(x1)
    return x1

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class TD3(object):

  def __init__(self, state_dim, action_dim, max_action):
    self.actor = Actor(state_dim, action_dim, max_action).to(device)
    self.actor_target = Actor(state_dim, action_dim, max_action).to(device)
    self.actor_target.load_state_dict(self.actor.state_dict())
    self.actor_optimizer = torch.optim.Adam(self.actor.parameters(),lr=0.0000001)
    self.critic = Critic(state_dim, action_dim).to(device)
    self.critic_target = Critic(state_dim, action_dim).to(device)
    self.critic_target.load_state_dict(self.critic.state_dict())
    self.critic_optimizer = torch.optim.Adam(self.critic.parameters(),lr=0.0001)
    self.max_action = max_action

  def select_action(self, state):
    state = torch.Tensor(state).to(device)
    #return self.actor(state).cpu().data.numpy().flatten()
    return self.actor(state).cpu().data.numpy()

  def train(self, replay_buffer, iterations, batch_size=100, discount=0.99, tau=0.005, policy_noise=0.2, noise_clip=0.5, policy_freq=2):

    for it in range(iterations):

      # Step 4: We sample a batch of transitions (s, s’, a, r) from the memory
      batch_states, batch_next_states, batch_actions, batch_rewards, batch_dones = replay_buffer.sample(batch_size)

      state = torch.Tensor(batch_states).to(device)
      next_state = torch.Tensor(batch_next_states).to(device)
      action = torch.Tensor(batch_actions).to(device)
      reward = torch.Tensor(batch_rewards).to(device)
      done = torch.Tensor(batch_dones).to(device)

      # Step 5: From the next state s’, the Actor target plays the next action a’
      next_action = self.actor_target(next_state)

      # Step 6: We add Gaussian noise to this next action a’ and we clamp it in a range of values supported by the environment
      noise = torch.Tensor(batch_actions).data.normal_(0, policy_noise).to(device)
      noise = noise.clamp(-noise_clip, noise_clip)
      next_action = (next_action + noise).clamp(-self.max_action, self.max_action)


      # Step 7: The two Critic targets take each the couple (s’, a’) as input and return two Q-values Qt1(s’,a’) and Qt2(s’,a’) as outputs
      target_Q1, target_Q2 = self.critic_target(next_state, torch.Tensor(next_action).to(device))

      # Step 8: We keep the minimum of these two Q-values: min(Qt1, Qt2)
      target_Q = torch.min(target_Q1, target_Q2)

      # Step 9: We get the final target of the two Critic models, which is: Qt = r + γ * min(Qt1, Qt2), where γ is the discount factor
      target_Q = reward + ((1 - done) * discount * target_Q).detach()

      # Step 10: The two Critic models take each the couple (s, a) as input and return two Q-values Q1(s,a) and Q2(s,a) as outputs
      current_Q1, current_Q2 = self.critic(state, action)

      # Step 11: We compute the loss coming from the two Critic models: Critic Loss = MSE_Loss(Q1(s,a), Qt) + MSE_Loss(Q2(s,a), Qt)
      critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)

      # Step 12: We backpropagate this Critic loss and update the parameters of the two Critic models with a SGD optimizer
      self.critic_optimizer.zero_grad()
      critic_loss.backward()
      self.critic_optimizer.step()

      # Step 13: Once every two iterations, we update our Actor model by performing gradient ascent on the output of the first Critic model
      if it % policy_freq == 0:
        actor_loss = -self.critic.Q1(state, self.actor(state)).mean()
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # Step 14: Still once every two iterations, we update the weights of the Actor target by polyak averaging
        for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
          target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

        # Step 15: Still once every two iterations, we update the weights of the Critic target by polyak averaging
        for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
          target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

  # Making a save method to save a trained model
  def save(self, filename, directory):
    torch.save(self.actor.state_dict(), '%s/%s_actor.pth' % (directory, filename))
    torch.save(self.critic.state_dict(), '%s/%s_critic.pth' % (directory, filename))

  # Making a load method to load a pre-trained model
  def load(self, filename, directory):
    self.actor.load_state_dict(torch.load('%s/%s_actor.pth' % (directory, filename)))
    self.critic.load_state_dict(torch.load('%s/%s_critic.pth' % (directory, filename)))

In [ ]:
def evaluate_policy(policy, env, access_point_id,eval_episodes=10):
  avg_reward = 0.
  for _ in range(eval_episodes):
    obs = env.reset()
    done = False
    while not done:
      action = policy.select_action(obs)
      action = env.reshape_action_space_from_model_to_dict(action)
      obs, reward, done, _ = env.step(action)
      avg_reward += reward#interp(sum(reward),[720000000,863000000],[0,1000])
  avg_reward /= eval_episodes
  print ("---------------------------------------")
  print ("Access Point: %i. Average Reward over the Evaluation Step: %f" % (access_point_id,avg_reward))
  print ("---------------------------------------")
  return avg_reward

In [ ]:
def training(policy,replay_buffer,env, access_point_id,access_point_radius,barrier):
    access_point_id = access_point_id
    #print('env')
    #print(env)
    env_name = "Access Point " + str(access_point_id) # Name of a environment (set it to any Continous environment you want)
    #print('env name')
    #print(env_name)
    seed = 0 # Random seed number
    start_timesteps = 50000 # Number of iterations/timesteps before which the model randomly chooses an action, and after which it starts to use the policy network
    start_timesteps_global = start_timesteps/2
    eval_freq = 5000 # How often the evaluation step is performed (after how many timesteps)
    max_timesteps = 500000 # Total number of iterations/timesteps
    save_models = True # Boolean checker whether or not to save the pre-trained model
    expl_noise = 0.1 # Exploration noise - STD value of exploration Gaussian noise
    batch_size = 100 # Size of the batch
    discount = 0.99 # Discount factor gamma, used in the calculation of the total discounted reward
    tau = 0.005 # Target network update rate
    policy_noise = 0.2 # STD of Gaussian noise added to the actions for the exploration purposes
    noise_clip = 0.5 # Maximum value of the Gaussian noise added to the actions (policy)
    policy_freq = 2 # Number of iterations to wait before the policy network (Actor model) is updated


    results_folder_name = "./results/access_point_%d" % (access_point_id)
    model_save_folder_name = "./pytorch_models/access_point_%d" % (access_point_id)
    dnn_model_save_folder_name = "./dnn_models/access_point_%d" % (access_point_id)

    if not os.path.exists(results_folder_name):
        os.makedirs(results_folder_name)
    if save_models and not os.path.exists(model_save_folder_name):
        os.makedirs(model_save_folder_name)
    if not os.path.exists(dnn_model_save_folder_name):
        os.makedirs(dnn_model_save_folder_name)

    file_name = "%s_%s_%s" % ("TD3", env_name, str(seed))
    file_name_dnn_model = "%s_%s_%d" % ("FL_DNN","Access_Point",access_point_id)
    file_name_0 = "Evaluations"
    file_name_1 = "timestep_rewards_energy_throughput"
    file_name_2 = "offloading_actions"
    file_name_3 = "power_actions"
    file_name_4 = "subcarrier_actions"
    file_name_5 = "allocated_RBs"
    file_name_6 = "fairnes_index"

    file_name_7 = "energy_efficiency_rewards"
    file_name_8 = "battery_energy_rewards"
    file_name_9 = "throughput_rewards"
    file_name_10 = "delay_rewards"
    file_name_11 = "sum_allocations_per_RB_matrix"
    file_name_12 = "RB_allocation_matrix"
    file_name_13 = "energy_rewards"
    file_name_14 = "delays"
    file_name_15 = "tasks_dropped"
    file_name_16 = "resource_allocation_matrix"
    file_name_17 = "resource_allocation_constraint_violation_count"
    file_name_18 = "global_reward"
    file_name_19 = "average_reward_in_memory_AP_%d"%(access_point_id)
    file_name_20 = "DNN_training_loss_AP_%d"%(access_point_id)

    evaluations = []#[evaluate_policy(policy, env, access_point_id, user_association_number)]
    total_timesteps = 0
    timesteps_since_eval = 0
    episode_num = 0
    done = True
    t0 = time.time()
    timestep_rewards = []
    timestep_rewards_energy_throughput = []
    offload_actions = []
    power_actions = []
    subcarrier_actions = []
    allocated_RBs = []
    fairness_index = []
    energy_efficiency_rewards = []
    battery_energy_rewards = []
    energy_rewards = []
    throughput_rewards = []
    delay_rewards = []
    sum_allocations_per_RB_matrix = []
    change_action = 0
    RB_allocation_matrix = []
    delays = []
    tasks_dropped = []
    resource_allocation_matrix = []
    resource_allocation_constraint_violation_count = []
    global_reward = []
    average_reward_in_memory = []
    # We start the main loop over 500,000 timesteps
    while total_timesteps < max_timesteps:

    # If the episode is done
        if done:
            # If we are not at the very beginning, we start the training process of the model
            if total_timesteps != 0:
                print("Access Point: {} Total Timesteps: {} Episode Num: {} Reward: {} User_Association_Reward: {} Average Reward in Training Memory: {}".format(access_point_id,total_timesteps, episode_num, env.episode_reward, env.user_association_channel_rate_reward, env.SBS.average_reward_in_memory))
                timestep_rewards.append([total_timesteps, episode_reward])
                timestep_rewards_energy_throughput.append([total_timesteps,episode_reward,env.total_energy,env.total_rate])
                offload_actions.append(env.offload_decisions)
                power_actions.append(env.powers)
                subcarrier_actions.append(env.subcarriers)
                allocated_RBs.append(env.Communication_Channel_1.allocated_RBs)
                fairness_index.append(env.SBS.fairness_index)

                energy_efficiency_rewards.append(env.SBS.energy_efficiency_rewards)
                battery_energy_rewards.append(env.SBS.battery_energy_rewards)
                throughput_rewards.append(env.SBS.throughput_rewards)
                delay_rewards.append(env.SBS.delay_rewards)
                sum_allocations_per_RB_matrix.append(env.sum_allocations_per_RB_matrix)
                RB_allocation_matrix.append(env.RB_allocation_matrix)
                energy_rewards.append(env.SBS.energy_rewards)
                delays.append(env.SBS.delays)
                tasks_dropped.append(env.SBS.tasks_dropped)
                resource_allocation_matrix.append(env.resource_block_allocation_matrix)
                resource_allocation_constraint_violation_count.append(env.resource_allocation_constraint_violation)
                average_reward_in_memory.append(env.SBS.average_reward_in_memory)

                policy.train(replay_buffer, episode_timesteps, batch_size, discount, tau, policy_noise, noise_clip, policy_freq)

            # We evaluate the episode and we save the policy
            if timesteps_since_eval >= eval_freq:
                timesteps_since_eval %= eval_freq
                evaluations.append(evaluate_policy(policy,env, access_point_id))
                policy.save(file_name, directory="./pytorch_models/access_point_%d" % (access_point_id))
                np.save("./results/access_point_%d/%s" % (access_point_id,file_name_0), evaluations)
                np.save("./results/access_point_%d/%s" % (access_point_id,file_name_1), timestep_rewards_energy_throughput)
                #np.save("./results/access_point_%d/%s" % (access_point_id,file_name_2), offload_actions)
                #np.save("./results/access_point_%d/%s" % (access_point_id,file_name_3), power_actions)
                #np.save("./results/access_point_%d/%s" % (access_point_id,file_name_4), subcarrier_actions)
                #np.save("./results/access_point_%d/%s" % (access_point_id,file_name_5), allocated_RBs)
                np.save("./results/access_point_%d/%s" % (access_point_id,file_name_6), fairness_index)
                np.save("./results/access_point_%d/%s" % (access_point_id,file_name_7), energy_efficiency_rewards)
                np.save("./results/access_point_%d/%s" % (access_point_id,file_name_8), battery_energy_rewards)
                np.save("./results/access_point_%d/%s" % (access_point_id,file_name_9), throughput_rewards)
                np.save("./results/access_point_%d/%s" % (access_point_id,file_name_10), delay_rewards)
                np.save("./results/access_point_%d/%s" % (access_point_id,file_name_11), sum_allocations_per_RB_matrix)
                np.save("./results/access_point_%d/%s" % (access_point_id,file_name_12), RB_allocation_matrix)
                np.save("./results/access_point_%d/%s" % (access_point_id,file_name_13), energy_rewards)
                np.save("./results/access_point_%d/%s" % (access_point_id,file_name_14), delays)
                np.save("./results/access_point_%d/%s" % (access_point_id,file_name_15), tasks_dropped)
                #np.save("./results/access_point_%d/%s" % (access_point_id,file_name_16), resource_allocation_matrix)
                np.save("./results/access_point_%d/%s" % (access_point_id,file_name_17), resource_allocation_constraint_violation_count)

            # When the training step is done, we reset the state of the environment
            if total_timesteps != 0:
                if total_timesteps >= 10000:#start_timesteps_global:
                    local_model = env.SBS.train_local_dnn()
                    np.save("./results/access_point_%d/%s" % (access_point_id,file_name_20), env.SBS.training_loss)
                    env.SBS.save_model(file_name_dnn_model,dnn_model_save_folder_name)
                    barrier.wait_for_aggregation(global_entity,local_model, access_point_id)
                else:
                    barrier.wait_for_aggregation(global_entity,env.SBS.access_point_model, access_point_id)

                barrier.wait_for_reassociations(env, global_entity, total_timesteps, user_association_episode_reward, access_point_radius)
                global_reward.append(global_entity.global_reward)
                #print('global_reward: ', global_entity.global_reward)
                np.save("./results/access_point_%d/%s" % (access_point_id,file_name_18), global_reward)
                user_association = global_entity.aggregate_user_associations()
                #print('aggregated association: ', user_association)
                #print('----------user_association-------------', user_association)
                #print('SBS label: ', env.SBS.SBS_label)
                env.SBS.reassociate_users(user_association)
                env.SBS.populate_buffer_memory_sample_with_reward(global_entity.global_reward)
                np.save("./results/access_point_%d/%s" % (access_point_id,file_name_19), average_reward_in_memory)
                global_entity.clear_local_models_memory()
                global_entity.clear_local_user_associations()
                global_entity.reset_global_reward()
            obs = env.reset()

            # Set the Done to False
            done = False

            # Set rewards and episode timesteps to zero
            episode_reward = 0
            user_association_episode_reward = 0
            episode_timesteps = 0
            episode_num += 1

        # Before 10000 timesteps, we play random actions
        if total_timesteps < start_timesteps:
            action = env.action_space.sample()
            action = env.enforce_constraint(action)
            action2, action = env.reshape_action_space_dict(action)

            new_obs, reward, done, _ = env.step(action)
            user_association_reward = env.user_association_channel_rate_reward

        else: # After 10000 timesteps, we switch to the model
            action = policy.select_action(np.array(obs))
            # If the explore_noise parameter is not 0, we add noise to the action and we clip it
            if expl_noise != 0:
                action = (action + np.random.normal(0, expl_noise, size=env.action_space.shape)).clip(env.action_space_low, env.action_space_high)

            #print(action)
            action = env.reshape_action_space_from_model_to_dict(action)
            reformed_action = env.apply_resource_allocation_constraint(action)

            new_obs, reward, done, _ = env.step(reformed_action)
            user_association_reward = env.user_association_channel_rate_reward

        #print("Action in training")
        #print(action)
        #print(' ')
        # The agent performs the action in the environment, then reaches the next state and receives the reward
        # We check if the episode is done
        done_bool = 0 if episode_timesteps + 1 == env.STEP_LIMIT else float(done)

        # We increase the total reward
        episode_reward += reward
        user_association_episode_reward+=user_association_reward
        #print('episode reward')
        #print(episode_reward)
        #episode_reward = interp(episode_reward,[720000000,863000000],[0,1000])
        # We store the new transition into the Experience Replay memory (ReplayBuffer)
        action = env.reshape_action_space_for_model(action)
        replay_buffer.add((obs, new_obs, action, reward, done_bool))

        # We update the state, the episode timestep, the total timesteps, and the timesteps since the evaluation of the policy
        obs = new_obs
        episode_timesteps += 1
        total_timesteps += 1
        timesteps_since_eval += 1

        # We add the last policy evaluation to our list of evaluations and we save our model
    evaluations.append(evaluate_policy(policy,env, access_point_id))
    if save_models: policy.save("%s" % (file_name), directory="./pytorch_models/access_point_%d" % (access_point_id))
    np.save("./results/access_point_%d/%s" % (access_point_id,file_name_0), evaluations)
    np.save("./results/access_point_%d/%s" % (access_point_id,file_name_1), timestep_rewards_energy_throughput)
    np.save("./results/access_point_%d/%s" % (access_point_id,file_name_2), offload_actions)
    np.save("./results/access_point_%d/%s" % (access_point_id,file_name_3), power_actions)
    np.save("./results/access_point_%d/%s" % (access_point_id,file_name_4), subcarrier_actions)
    np.save("./results/access_point_%d/%s" % (access_point_id,file_name_5), allocated_RBs)
    np.save("./results/access_point_%d/%s" % (access_point_id,file_name_6), fairness_index)
    np.save("./results/access_point_%d/%s" % (access_point_id,file_name_7), energy_efficiency_rewards)
    np.save("./results/access_point_%d/%s" % (access_point_id,file_name_8), battery_energy_rewards)
    np.save("./results/access_point_%d/%s" % (access_point_id,file_name_9), throughput_rewards)
    np.save("./results/access_point_%d/%s" % (access_point_id,file_name_10), delay_rewards)
    np.save("./results/access_point_%d/%s" % (access_point_id,file_name_11), sum_allocations_per_RB_matrix)
    np.save("./results/access_point_%d/%s" % (access_point_id,file_name_12), RB_allocation_matrix)
    np.save("./results/access_point_%d/%s" % (access_point_id,file_name_13), energy_rewards)
    np.save("./results/access_point_%d/%s" % (access_point_id,file_name_14), delays)
    np.save("./results/access_point_%d/%s" % (access_point_id,file_name_15), tasks_dropped)
    #np.save("./results/access_point_%d/%s" % (access_point_id,file_name_16), resource_allocation_matrix)
    np.save("./results/access_point_%d/%s" % (access_point_id,file_name_17), resource_allocation_constraint_violation_count)


In [ ]:
barrier = CustomBarrier(num_access_points)

In [ ]:
access_points_envs = []
access_point_users = []
#user_association_epoch_number = i
policies = []
replay_buffers = []
threadlist = []

state_dim = 0
action_dim = 0
max_action = 0
seed = 0

In [ ]:
for x in range(1,num_access_points+1):
    env_name = "AccessPoint" + str(x) + "-v0"
    env = NetworkEnv(all_users,access_points[x-1],access_point_coordinates,access_point_radius)        
    access_points_envs.append(env)

In [ ]:
for x in range(0,num_access_points):
    torch.manual_seed(seed)
    np.random.seed(seed)
    state_dim = access_points_envs[x].observation_space_dim
    action_dim = access_points_envs[x].action_space_dim
    max_action = float(access_points_envs[x].box_action_space.high[0][1]) # to change this soon

    policy = TD3(state_dim, action_dim, max_action)
    replay_buffer = ReplayBuffer()

    policies.append(policy)
    replay_buffers.append(replay_buffer)

In [ ]:
for x in range(0,num_access_points):
    t = threading.Thread(target=training,args=(policies[x],replay_buffers[x],access_points_envs[x],x+1,access_point_radius,barrier))
    print('inner thread id: ', t.name)
    threadlist.append(t)
    #t.start()

for t in threadlist:
    t.start()
for tr in threadlist:
    tr.join()

## Inference

In [ ]:
initial_user_associations = global_entity.perform_random_association(all_users)

In [ ]:
for access_point in access_points:
   access_point_users = []
   for user in all_users:
      for association in initial_user_associations:
         #print(association)
         if association[0] == user.user_label and association[1][0] == access_point.SBS_label:
            user.distance_from_associated_access_point = association[1][1]
            access_point_users.append(user)

   access_point.associate_users(access_point_users)

In [ ]:
access_points_envs = []
for x in range(1,num_access_points+1):
    env_name = "AccessPoint" + str(x) + "-v0"
    env = NetworkEnv(all_users,access_points[x-1],access_point_coordinates,access_point_radius)        
    access_points_envs.append(env)

In [ ]:
seed = 0
access_point_id = 1
for x in range(0,num_access_points):
    torch.manual_seed(seed)
    np.random.seed(seed)
    model_save_folder_name = "./pytorch_models/access_point_%d" % (access_point_id)
    env_name = "Access Point " + str(access_point_id)
    file_name = "%s_%s_%s" % ("TD3", env_name, str(seed))
    state_dim = access_points_envs[x].observation_space_dim
    action_dim = access_points_envs[x].action_space_dim
    max_action = float(access_points_envs[x].box_action_space.high[0][1]) # to change this soon

    policy = TD3(state_dim, action_dim, max_action)
    policy.load(file_name, model_save_folder_name)
    replay_buffer = ReplayBuffer()

    policies.append(policy)
    replay_buffers.append(replay_buffer)
    access_point_id+=1

In [ ]:
access_point_id = 1
for access_point in access_points:
    file_name_dnn_model = "%s_%s_%d" % ("FL_DNN","Access_Point",access_point_id)
    dnn_model_save_folder_name = "./dnn_models/access_point_%d" % (access_point_id)
    access_points_envs[access_point_id-1].SBS.load_model(file_name_dnn_model,dnn_model_save_folder_name)
    access_point_id+=1

In [ ]:
SBS_association = []
timestep_counter = 0
global_entity.local_associations.clear()
access_point_id = 1
for access_point in access_points:
    SBS_association = access_points_envs[access_point_id-1].SBS.predict_future_association(access_point_radius, timestep_counter,  access_points_envs[access_point_id-1].eMBB_Users,  access_points_envs[access_point_id-1].URLLC_Users)
    global_entity.acquire_local_user_associations(SBS_association)
    access_point_id+=1

user_association = global_entity.aggregate_user_associations()

access_point_id = 1
for access_point in access_points:
    access_points_envs[access_point_id-1].SBS.reassociate_users(user_association)
    access_point_id+=1

In [ ]:
def evaluate_policy(policy, env ,access_point_id, eval_episodes=1):
  inference_results_folder_name = "./inference_results/access_point_%d" % (access_point_id)
  if not os.path.exists(inference_results_folder_name):
    os.makedirs(inference_results_folder_name)
  avg_reward = 0.
  for _ in range(eval_episodes):
    obs = env.reset()
    done = False
    while not done:
      action = policy.select_action(obs)
      action = env.reshape_action_space_from_model_to_dict(action)
      reformed_action = env.apply_resource_allocation_constraint(action)
      obs, reward, done, _ = env.step(reformed_action)
      avg_reward += reward
  avg_reward /= eval_episodes
  env.SBS.collect_user_results()
  file_name_1 = "users_achieved_channel_rates_AP_%d"%(access_point_id)
  file_name_2 = "users_distances_to_associated_APs_AP_%d"%(access_point_id)
  file_name_3 = "users_distances_to_other_APs_AP_%d"%(access_point_id)
  file_name_4 = "users_channel_rates_to_other_APs_AP_%d"%(access_point_id)
  file_name_5 = "access_point_users_AP_%d"%(access_point_id)
  np.save("./inference_results/access_point_%d/%s" % (access_point_id,file_name_1), env.SBS.users_achieved_channel_rates)
  np.save("./inference_results/access_point_%d/%s" % (access_point_id,file_name_2), env.SBS.users_distances_to_associated_APs)
  np.save("./inference_results/access_point_%d/%s" % (access_point_id,file_name_3), env.SBS.users_distances_to_other_APs)
  np.save("./inference_results/access_point_%d/%s" % (access_point_id,file_name_4), env.SBS.users_channel_rates_to_other_APs)
  np.save("./inference_results/access_point_%d/%s" % (access_point_id,file_name_5), env.SBS.access_point_users)

In [ ]:
access_point_id = 1
for access_point in access_points:
    evaluate_policy(policies[access_point_id-1],access_points_envs[access_point_id-1], access_point_id)
    access_point_id+=1